In [1]:
# procedure for calculating hydraulic confinement along a pressure tunnel 
# completed procedure will in the form of a design report
#   safety factor against hydraulic confinement calculated at stationed points along tunnel alignment
#   calculation determines the minimum dstance from the stationed point to the terrain surface 

# project name: 'Nam Ang' 

In [2]:
# ToDo
#   DONE need to read in 'slope' and get slope values for all buffer grid points 'MIN'
#   DONE Nam Ang headrace tunnel alignment and DTM
#   DONE add calculation for hydraulic confinement safety factor (SF)
#   add results plot for hydraulic confinement safety factor
#   put common functions (e.g. setup, data format conversions) into a separate module/API
#   DONE use Nam Ang HEP data for initial testing and demo
#   provide full desgin criteria text in Jupyter Notebook design report
#   DONE add check plot with alignment, station points, buffer points
#   DONE add check plot with MIN buffer grid points
#   refactor code

In [3]:
# import required python libraires
import numpy as np
from numpy import *
import pandas as pd
import geopandas as gpd
import shapely as sp
import matplotlib.pylab as pylab
import matplotlib.pyplot as plt
%matplotlib inline

# testing
#import random
# check pd and gpd version
#print(pd.__version__)
#print(gpd.__version__)

In [4]:
# python setup for qgis processing
import sys, os
from qgis.core import QgsApplication
from PyQt4.QtGui import QApplication
app = QApplication([], True)  #True -> window display enabled
QgsApplication.setPrefixPath("/usr", True)
QgsApplication.initQgis()
sys.path.append('/usr/share/qgis/python/plugins')  #export PYTHONPATH not needed in start script
from processing.core.Processing import Processing
Processing.initialize() 
import processing
#from processing.tools import *  #not needed currently

In [5]:
# set up plotly in 'offline' mode                            #ToDo JK: this should be cleaned up and documented
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot,iplot
import plotly.graph_objs as go
init_notebook_mode(connected=True)
print (__version__) # requires version >= 1.9.0

# collect all plotly setups here
import plotly.offline as plotly
from plotly.graph_objs import *

2.2.1


In [6]:
# import DSS modules
#import geospatial as geo

In [7]:
# set wd for this procedure and project 
os.chdir("/home/kaelin_joseph/DSS.HydraulicConfinement/")

In [8]:
# define required input files
DTM = "data/in/NamAngTopogRaster.tif"  #DEM with surface topography
Alignment = "data/in/NamAngAlignmentHRT.r1.csv"  #tunnel alignment
#   Alignment contains fields "Point","Type","Station","Northing","Easting","Elevation"
#   check Alignment data: no spaces between comma-separated fields
#   check Alignment data: no trailing blank lines, no duplicate lines
DTM_slope='data/in/NamAngTopogSlopeRaster.tif'  #DEM containing slope angle as attribute

In [9]:
# define required input data

# mapping
crs = {'init': 'epsg:32648'}  #define crs for project
grass_region = "726000,729500,1670500,1674000"  #map region E1,E2,N1,N2

# resolution for analysis
grass_station_dist = 20  #target station interval
#grass_station_dist = 50  #testing
c = 0.5  #ring buffer radius = c*h  (h=overburden)
res = 3.0  #nominal ring buffer grid resolution 
#res = 25.0  #testing

# rock properties
density_rock = 28.0  #kN/m3

# hyrdaulic properties
max_static_water_level = 637.20 #MASL 

In [10]:
# define temporary data
Alignment_shp ='tmp/NamAngAlignment.shp'  #alignment shp from Alignment
Alignment_grass_csv = 'tmp/NamAngAlignmentGrass.csv'  #alignment csv fixed for grass
Alignment_line_shp = "tmp/NamAngAlignmentLine.shp"  #intermediate data
Alignment_stationed_shp = "tmp/NamAngAlignmentStationed.shp"  #alignment shp containing station points
Alignment_dtm_csv = "tmp/NamAngAlignmentDTM.csv"  #alignment including terrain elevations at station points
Buffer_shp = "tmp/NamAngBuffer.shp"  #buffer shp containing ring grid points at a particular station point
Buffer_all_csv = "tmp/NamAngBufferAll.csv"  # all station point ring buffers written to csv
Buffer_all_shp = "tmp/NamAngBufferFinal.shp"
Buffer_dtm_csv = "tmp/NamAngBufferDTM.csv"
Buffer_slope_csv = "tmp/NamAngBufferSlope.csv"

In [11]:
# define output files


In [12]:
# create alignment_df (dataframe) from Alignment csv                     #ToDo JK: make into csv -> df function
alignment_df = pd.read_csv(Alignment)

# delete row if only NA are present in row
alignment_df = alignment_df.dropna(how = "all")
# round alignment_df to three decimals
alignment_df = alignment_df.round(decimals=3)

In [13]:
alignment_df

Point Type   Station    Northing    Easting  Elevation
0      0  BEG     0+000  1673035.25  726651.46    625.850
1      1   PT  0+163.81  1673051.68  726815.60    625.543
2      2   PT  2+063.77  1672130.47  728479.47    504.226
3      3   PT  2+603.58  1671662.07  728758.97    469.758
4      4  END  3+030.73  1671268.20  728581.38    464.220

In [14]:
# create stationed alignment as Alignment_stationed_shp                 #ToDo JK: make into stationing function
# from points in alignment_df
#   required grass input data: x, y coordinates at each alignment point
#   grass input for function v.in.line 1) must have x first and y second; 2) no header 
#   grass output: x, y coordinates at each station point along alignment
alignment_df_grass = alignment_df.loc[:,["Easting", "Northing"]]  #x first and y second
# write Alignment_grass_csv
alignment_df_grass.to_csv(Alignment_grass_csv, header=False, index=False)  #no header

# points to line, write output to Alignment_line_shp
processing.runalg("grass7:v.in.lines",Alignment_grass_csv,",",False,
                  grass_region,0,Alignment_line_shp)  #no spaces between commas

# line to station points, ouput segmented polyline to Alignment_stationed_shp
processing.runalg("grass7:v.to.points",Alignment_line_shp,grass_station_dist,1,True,
                  grass_region,-1,0.0001,0,Alignment_stationed_shp)  #no spaces between commas

{'output': 'tmp/NamAngAlignmentStationed.shp'}

In [15]:
# create alignment_stationed_df from Alignment_stationed_shp                #ToDo JK: make into shp -> df function
#   required output data: x_align, y_align at each station point
alignment_stationed_df = gpd.read_file(Alignment_stationed_shp)

# create columns for x_align, y_align, then delete columns cat_ and geometry
alignment_stationed_df["x_align"] = alignment_stationed_df.geometry.x
alignment_stationed_df["y_align"] = alignment_stationed_df.geometry.y
alignment_stationed_df = alignment_stationed_df.drop(columns =["cat_", "geometry"])

In [16]:
alignment_stationed_df

x_align       y_align
0    726651.460000  1.673035e+06
1    726669.697778  1.673037e+06
2    726687.935556  1.673039e+06
3    726706.173333  1.673041e+06
4    726724.411111  1.673043e+06
5    726742.648889  1.673044e+06
6    726760.886667  1.673046e+06
7    726779.124444  1.673048e+06
8    726797.362222  1.673050e+06
9    726815.600000  1.673052e+06
10   726832.931979  1.673042e+06
11   726850.263958  1.673032e+06
12   726867.595938  1.673023e+06
13   726884.927917  1.673013e+06
14   726902.259896  1.673004e+06
15   726919.591875  1.672994e+06
16   726936.923854  1.672985e+06
17   726954.255833  1.672975e+06
18   726971.587812  1.672965e+06
19   726988.919792  1.672956e+06
20   727006.251771  1.672946e+06
21   727023.583750  1.672937e+06
22   727040.915729  1.672927e+06
23   727058.247708  1.672917e+06
24   727075.579688  1.672908e+06
25   727092.911667  1.672898e+06
26   727110.243646  1.672889e+06
27   727127.575625  1.672879e+06
28   727144.907604  1.672869e+06
29   727162.239583  1.672860e+06
..             ...           ...
126  728689.095000  1.671779e+06
127  728699.077143  1.671762e+06
128  728709.059286  1.671746e+06
129  728719.041429  1.671729e+06
130  728729.023571  1.671712e+06
131  728739.005714  1.671696e+06
132  728748.987857  1.671679e+06
133  728758.970000  1.671662e+06
134  728750.897727  1.671644e+06
135  728742.825455  1.671626e+06
136  728734.753182  1.671608e+06
137  728726.680909  1.671590e+06
138  728718.608636  1.671573e+06
139  728710.536364  1.671555e+06
140  728702.464091  1.671537e+06
141  728694.391818  1.671519e+06
142  728686.319545  1.671501e+06
143  728678.247273  1.671483e+06
144  728670.175000  1.671465e+06
145  728662.102727  1.671447e+06
146  728654.030455  1.671429e+06
147  728645.958182  1.671411e+06
148  728637.885909  1.671394e+06
149  728629.813636  1.671376e+06
150  728621.741364  1.671358e+06
151  728613.669091  1.671340e+06
152  728605.596818  1.671322e+06
153  728597.524545  1.671304e+06
154  728589.452273  1.671286e+06
155  728581.380000  1.671268e+06

[156 rows x 2 columns]

In [17]:
# add required fields to alignment_stationed_df

# add "id" 
alignment_stationed_df["id_point"] = alignment_stationed_df.index

# add "distance_stat" referencing length between point n and point n+1
#   is this what is intended??                                                                       ToDo: JK
alignment_stationed_df["distance_stat"] = np.nan
for n in range(0, len(alignment_stationed_df)-1):
    alignment_stationed_df.iloc[n, alignment_stationed_df.columns.get_loc("distance_stat")] = (
        ((alignment_stationed_df.iloc[n +1]["x_align"] - alignment_stationed_df.iloc[n]["x_align"])**2
        +(alignment_stationed_df.iloc[n +1]["y_align"] - alignment_stationed_df.iloc[n]["y_align"])**2 )**(0.5) )

# add "distance_stat_sum"
alignment_stationed_df["distance_stat_sum"] = np.nan
####alignment_stationed_df.iloc[0, alignment_stationed_df.columns.get_loc("distance_stat_sum")] = 0.0
for n in range(0, len(alignment_stationed_df) -1):
    distance = ( alignment_stationed_df.loc[(alignment_stationed_df.id_point.isin(range(0,n +1))), 
                        "distance_stat"] )
    distances = distance.tolist()
    alignment_stationed_df.iloc[n, alignment_stationed_df.columns.get_loc("distance_stat_sum")] = (
                                                                                        sum(distances) )
    
#alignment_stationed_df.head()
alignment_stationed_df

x_align       y_align  id_point  distance_stat  distance_stat_sum
0    726651.460000  1.673035e+06         0      18.328917          18.328917
1    726669.697778  1.673037e+06         1      18.328917          36.657834
2    726687.935556  1.673039e+06         2      18.328917          54.986750
3    726706.173333  1.673041e+06         3      18.328917          73.315667
4    726724.411111  1.673043e+06         4      18.328917          91.644584
5    726742.648889  1.673044e+06         5      18.328917         109.973501
6    726760.886667  1.673046e+06         6      18.328917         128.302418
7    726779.124444  1.673048e+06         7      18.328917         146.631334
8    726797.362222  1.673050e+06         8      18.328917         164.960251
9    726815.600000  1.673052e+06         9      19.811096         184.771347
10   726832.931979  1.673042e+06        10      19.811096         204.582443
11   726850.263958  1.673032e+06        11      19.811096         224.393539
12   726867.595938  1.673023e+06        12      19.811096         244.204635
13   726884.927917  1.673013e+06        13      19.811096         264.015730
14   726902.259896  1.673004e+06        14      19.811096         283.826826
15   726919.591875  1.672994e+06        15      19.811096         303.637922
16   726936.923854  1.672985e+06        16      19.811096         323.449018
17   726954.255833  1.672975e+06        17      19.811096         343.260114
18   726971.587812  1.672965e+06        18      19.811096         363.071210
19   726988.919792  1.672956e+06        19      19.811096         382.882306
20   727006.251771  1.672946e+06        20      19.811096         402.693401
21   727023.583750  1.672937e+06        21      19.811096         422.504497
22   727040.915729  1.672927e+06        22      19.811096         442.315593
23   727058.247708  1.672917e+06        23      19.811096         462.126689
24   727075.579688  1.672908e+06        24      19.811096         481.937785
25   727092.911667  1.672898e+06        25      19.811096         501.748881
26   727110.243646  1.672889e+06        26      19.811096         521.559976
27   727127.575625  1.672879e+06        27      19.811096         541.371072
28   727144.907604  1.672869e+06        28      19.811096         561.182168
29   727162.239583  1.672860e+06        29      19.811096         580.993264
..             ...           ...       ...            ...                ...
126  728689.095000  1.671779e+06       126      19.480459        2495.395547
127  728699.077143  1.671762e+06       127      19.480459        2514.876006
128  728709.059286  1.671746e+06       128      19.480459        2534.356465
129  728719.041429  1.671729e+06       129      19.480459        2553.836924
130  728729.023571  1.671712e+06       130      19.480459        2573.317383
131  728739.005714  1.671696e+06       131      19.480459        2592.797841
132  728748.987857  1.671679e+06       132      19.480459        2612.278300
133  728758.970000  1.671662e+06       133      19.638877        2631.917178
134  728750.897727  1.671644e+06       134      19.638877        2651.556055
135  728742.825455  1.671626e+06       135      19.638877        2671.194933
136  728734.753182  1.671608e+06       136      19.638877        2690.833810
137  728726.680909  1.671590e+06       137      19.638877        2710.472687
138  728718.608636  1.671573e+06       138      19.638877        2730.111565
139  728710.536364  1.671555e+06       139      19.638877        2749.750442
140  728702.464091  1.671537e+06       140      19.638877        2769.389320
141  728694.391818  1.671519e+06       141      19.638877        2789.028197
142  728686.319545  1.671501e+06       142      19.638877        2808.667075
143  728678.247273  1.671483e+06       143      19.638877        2828.305952
144  728670.175000  1.671465e+06       144      19.638877        2847.944829
145  728662.102727  1.671447e+06       145      19.638877        2867.583707
146  72

In [18]:
# add required field "distance_intermed_align" to alignment_df
alignment_df["distance_intermed_align"] = np.nan

for n in range(0, len(alignment_df) -1):
    alignment_df.iloc[n, alignment_df.columns.get_loc("distance_intermed_align")] = (
        ((alignment_df.iloc[n +1]["Easting"]-alignment_df.iloc[n]["Easting"])**2 
             +(alignment_df.iloc[n +1]["Northing"]-alignment_df.iloc[n]["Northing"])**2 )**(0.5) )
    
#alignment_df.head()
alignment_df

Point Type   Station    Northing    Easting  Elevation  \
0      0  BEG     0+000  1673035.25  726651.46    625.850   
1      1   PT  0+163.81  1673051.68  726815.60    625.543   
2      2   PT  2+063.77  1672130.47  728479.47    504.226   
3      3   PT  2+603.58  1671662.07  728758.97    469.758   
4      4  END  3+030.73  1671268.20  728581.38    464.220   

   distance_intermed_align  
0               164.960251  
1              1901.865201  
2               545.452849  
3               432.055303  
4                      NaN

In [19]:
# join alignment_df to alignment_stationed_df
alignment_stationed_df = pd.merge(left= alignment_stationed_df, right = alignment_df, 
                 left_on = ["x_align","y_align"], 
                 right_on = ["Easting","Northing"], how = "left")

# clean up alignment_stationed_df
try:
    alignment_stationed_df = (
        alignment_stationed_df.drop(columns =["Point", "Type", "Northing", "Easting"]) )
except:
    pass

#alignment_stationed_df.head()
alignment_stationed_df

x_align       y_align  id_point  distance_stat  distance_stat_sum  \
0    726651.460000  1.673035e+06         0      18.328917          18.328917   
1    726669.697778  1.673037e+06         1      18.328917          36.657834   
2    726687.935556  1.673039e+06         2      18.328917          54.986750   
3    726706.173333  1.673041e+06         3      18.328917          73.315667   
4    726724.411111  1.673043e+06         4      18.328917          91.644584   
5    726742.648889  1.673044e+06         5      18.328917         109.973501   
6    726760.886667  1.673046e+06         6      18.328917         128.302418   
7    726779.124444  1.673048e+06         7      18.328917         146.631334   
8    726797.362222  1.673050e+06         8      18.328917         164.960251   
9    726815.600000  1.673052e+06         9      19.811096         184.771347   
10   726832.931979  1.673042e+06        10      19.811096         204.582443   
11   726850.263958  1.673032e+06        11      19.811096         224.393539   
12   726867.595938  1.673023e+06        12      19.811096         244.204635   
13   726884.927917  1.673013e+06        13      19.811096         264.015730   
14   726902.259896  1.673004e+06        14      19.811096         283.826826   
15   726919.591875  1.672994e+06        15      19.811096         303.637922   
16   726936.923854  1.672985e+06        16      19.811096         323.449018   
17   726954.255833  1.672975e+06        17      19.811096         343.260114   
18   726971.587812  1.672965e+06        18      19.811096         363.071210   
19   726988.919792  1.672956e+06        19      19.811096         382.882306   
20   727006.251771  1.672946e+06        20      19.811096         402.693401   
21   727023.583750  1.672937e+06        21      19.811096         422.504497   
22   727040.915729  1.672927e+06        22      19.811096         442.315593   
23   727058.247708  1.672917e+06        23      19.811096         462.126689   
24   727075.579688  1.672908e+06        24      19.811096         481.937785   
25   727092.911667  1.672898e+06        25      19.811096         501.748881   
26   727110.243646  1.672889e+06        26      19.811096         521.559976   
27   727127.575625  1.672879e+06        27      19.811096         541.371072   
28   727144.907604  1.672869e+06        28      19.811096         561.182168   
29   727162.239583  1.672860e+06        29      19.811096         580.993264   
..             ...           ...       ...            ...                ...   
126  728689.095000  1.671779e+06       126      19.480459        2495.395547   
127  728699.077143  1.671762e+06       127      19.480459        2514.876006   
128  728709.059286  1.671746e+06       128      19.480459        2534.356465   
129  728719.041429  1.671729e+06       129      19.480459        2553.836924   
130  728729.023571  1.671712e+06       130      19.480459        2573.317383   
131  728739.005714  1.671696e+06       131      19.480459        2592.797841   
132  728748.987857  1.671679e+06       132      19.480459        2612.278300   
133  728758.970000  1.671662e+06       133      19.638877        2631.917178   
134  728750.897727  1.671644e+06       134      19.638877        2651.556055   
135  728742.825455  1.671626e+06       135      19.638877        2671.194933   
136  728734.753182  1.671608e+06       136      19.638877        2690.833810   
137  728726.680909  1.671590e+06       137      19.638877        2710.472687   
138  728718.608636  1.671573e+06       138      19.638877        2730.111565   
139  728710.536364  1.671555e+06       139      19.638877        2749.750442   
140  728702.464091  1.671537e+06       140      19.638877        2769.389320   
141  728694.391818  1.671519e+06       141      19.638877        2789.028197   
142  728686.319545  1.671501e+06       142      19.638877        2808.667075   
143  728678.247273  1.671483e+06       143      19.638877        2828.305952   
144  728670

In [20]:
# get id_points for alignment points in alignment_stationed_df 
#   select points where Elevation of point not NaN
id_points_align =  (
    alignment_stationed_df.loc[(alignment_stationed_df.Elevation.isin(alignment_df["Elevation"])), "id_point"] )
id_points_align= id_points_align.tolist()
id_points_align                                                                         

[0, 9, 105, 133, 155]

In [21]:
len(id_points_align)

5

In [22]:
# prepare intermediated data in alignment_stationed_df required to interpolate alignment elevations

# fill in "Elevation" and "distance_intermed_align" for points in alignment_stationed_df 
#   where points of alignment_points_df != alignment_df

# why is this needed??                                                                                ToDo: JK
for n in range(0, len(id_points_align) -1): 
    alignment_stationed_df.loc[(alignment_stationed_df.id_point.isin(range(id_points_align[n] +1, 
                                id_points_align[n +1]))), "Elevation"] = ( 
                                                                    alignment_df["Elevation"][n] )

for n in range(0, len(id_points_align) -1): 
    alignment_stationed_df.loc[(alignment_stationed_df.id_point.isin(range(id_points_align[n] +1, 
                                id_points_align[n +1]))), "distance_intermed_align"] = ( 
                                                        alignment_df["distance_intermed_align"][n] )

# add "distance_intermed_stat" to alignment_stationed_df 
alignment_stationed_df["distance_intermed_stat"] = np.nan

for n in range(0, 1):
    alignment_stationed_df.loc[(alignment_stationed_df.id_point.isin(range(id_points_align[n], 
                                id_points_align[n +1]))), "distance_intermed_stat"] =  ( 
                                                    alignment_stationed_df["distance_stat_sum"] )
### for n in range(1, len(id_points_align) -1): 
###     alignment_stationed_df.loc[(alignment_stationed_df.id_point.isin(range(id_points_align[n], 
###                                 id_points_align[n +1]))), "distance_intermed_stat"] = ( 
###                                                     alignment_stationed_df["distance_stat_sum"] - 
###                                                     alignment_df["distance_intermed_align"][n -1] )
for n in range(1, len(id_points_align) -1):
      alignment_stationed_df.loc[(alignment_stationed_df.id_point.isin(range(id_points_align[n], 
                                  id_points_align[n +1]))), "distance_intermed_stat"] = ( 
                                             alignment_stationed_df["distance_stat_sum"] - 
                                             alignment_stationed_df["distance_stat_sum"][id_points_align[n] -1] )
    
#alignment_stationed_df.head()
alignment_stationed_df

x_align       y_align  id_point  distance_stat  distance_stat_sum  \
0    726651.460000  1.673035e+06         0      18.328917          18.328917   
1    726669.697778  1.673037e+06         1      18.328917          36.657834   
2    726687.935556  1.673039e+06         2      18.328917          54.986750   
3    726706.173333  1.673041e+06         3      18.328917          73.315667   
4    726724.411111  1.673043e+06         4      18.328917          91.644584   
5    726742.648889  1.673044e+06         5      18.328917         109.973501   
6    726760.886667  1.673046e+06         6      18.328917         128.302418   
7    726779.124444  1.673048e+06         7      18.328917         146.631334   
8    726797.362222  1.673050e+06         8      18.328917         164.960251   
9    726815.600000  1.673052e+06         9      19.811096         184.771347   
10   726832.931979  1.673042e+06        10      19.811096         204.582443   
11   726850.263958  1.673032e+06        11      19.811096         224.393539   
12   726867.595938  1.673023e+06        12      19.811096         244.204635   
13   726884.927917  1.673013e+06        13      19.811096         264.015730   
14   726902.259896  1.673004e+06        14      19.811096         283.826826   
15   726919.591875  1.672994e+06        15      19.811096         303.637922   
16   726936.923854  1.672985e+06        16      19.811096         323.449018   
17   726954.255833  1.672975e+06        17      19.811096         343.260114   
18   726971.587812  1.672965e+06        18      19.811096         363.071210   
19   726988.919792  1.672956e+06        19      19.811096         382.882306   
20   727006.251771  1.672946e+06        20      19.811096         402.693401   
21   727023.583750  1.672937e+06        21      19.811096         422.504497   
22   727040.915729  1.672927e+06        22      19.811096         442.315593   
23   727058.247708  1.672917e+06        23      19.811096         462.126689   
24   727075.579688  1.672908e+06        24      19.811096         481.937785   
25   727092.911667  1.672898e+06        25      19.811096         501.748881   
26   727110.243646  1.672889e+06        26      19.811096         521.559976   
27   727127.575625  1.672879e+06        27      19.811096         541.371072   
28   727144.907604  1.672869e+06        28      19.811096         561.182168   
29   727162.239583  1.672860e+06        29      19.811096         580.993264   
..             ...           ...       ...            ...                ...   
126  728689.095000  1.671779e+06       126      19.480459        2495.395547   
127  728699.077143  1.671762e+06       127      19.480459        2514.876006   
128  728709.059286  1.671746e+06       128      19.480459        2534.356465   
129  728719.041429  1.671729e+06       129      19.480459        2553.836924   
130  728729.023571  1.671712e+06       130      19.480459        2573.317383   
131  728739.005714  1.671696e+06       131      19.480459        2592.797841   
132  728748.987857  1.671679e+06       132      19.480459        2612.278300   
133  728758.970000  1.671662e+06       133      19.638877        2631.917178   
134  728750.897727  1.671644e+06       134      19.638877        2651.556055   
135  728742.825455  1.671626e+06       135      19.638877        2671.194933   
136  728734.753182  1.671608e+06       136      19.638877        2690.833810   
137  728726.680909  1.671590e+06       137      19.638877        2710.472687   
138  728718.608636  1.671573e+06       138      19.638877        2730.111565   
139  728710.536364  1.671555e+06       139      19.638877        2749.750442   
140  728702.464091  1.671537e+06       140      19.638877        2769.389320   
141  728694.391818  1.671519e+06       141      19.638877        2789.028197   
142  728686.319545  1.671501e+06       142      19.638877        2808.667075   
143  728678.247273  1.671483e+06       143      19.638877        2828.305952   
144  728670

In [23]:
# interpolate alignment elevation ("z_align") at all station points and write to alignment_stationed_df

# add variable "z_align" to alignment_stationed_df
alignment_stationed_df["z_align"] = np.nan

for i in range(0, len(alignment_stationed_df)):
    # alignment points
    if i in id_points_align:
        alignment_stationed_df.iloc[i, alignment_stationed_df.columns.get_loc("z_align")] = ( 
                                                        alignment_stationed_df.iloc[i]["Elevation"] )
    # stationed points
    else:
        id_points_align_plus_point_n = id_points_align
        id_points_align_plus_point_n.append(i)
        id_points_align_plus_point_n.sort()
        m = id_points_align_plus_point_n.index(i) +1  #index of point n +1 (next alignment point)
        n = id_points_align_plus_point_n[m]  #id_point of next alignment point  
        o = id_points_align_plus_point_n.index(i) -1  #index of point n -1 (previous alignment point)
        p = id_points_align_plus_point_n[o]  #id_point of previous alignment point
        
        alignment_stationed_df.iloc[i, alignment_stationed_df.columns.get_loc("z_align")] = ( 
                                                    alignment_stationed_df.iloc[p]["Elevation"] 
                                                    +(alignment_stationed_df.iloc[n]["Elevation"] 
                                                    -alignment_stationed_df.iloc[p]["Elevation"]) 
                                                        /alignment_stationed_df.iloc[i]["distance_intermed_align"]
                                                        *alignment_stationed_df.iloc[i-1]["distance_intermed_stat"] )

        id_points_align_plus_point_n.remove(i)  #needed ??                                           #ToDo JK

alignment_stationed_df = alignment_stationed_df.drop(columns = ["distance_intermed_align"])
alignment_stationed_df = alignment_stationed_df.drop(columns = ["distance_intermed_stat"])
alignment_stationed_df = alignment_stationed_df.drop(columns = ["Elevation"])

#alignment_stationed_df.head()
alignment_stationed_df

x_align       y_align  id_point  distance_stat  distance_stat_sum  \
0    726651.460000  1.673035e+06         0      18.328917          18.328917   
1    726669.697778  1.673037e+06         1      18.328917          36.657834   
2    726687.935556  1.673039e+06         2      18.328917          54.986750   
3    726706.173333  1.673041e+06         3      18.328917          73.315667   
4    726724.411111  1.673043e+06         4      18.328917          91.644584   
5    726742.648889  1.673044e+06         5      18.328917         109.973501   
6    726760.886667  1.673046e+06         6      18.328917         128.302418   
7    726779.124444  1.673048e+06         7      18.328917         146.631334   
8    726797.362222  1.673050e+06         8      18.328917         164.960251   
9    726815.600000  1.673052e+06         9      19.811096         184.771347   
10   726832.931979  1.673042e+06        10      19.811096         204.582443   
11   726850.263958  1.673032e+06        11      19.811096         224.393539   
12   726867.595938  1.673023e+06        12      19.811096         244.204635   
13   726884.927917  1.673013e+06        13      19.811096         264.015730   
14   726902.259896  1.673004e+06        14      19.811096         283.826826   
15   726919.591875  1.672994e+06        15      19.811096         303.637922   
16   726936.923854  1.672985e+06        16      19.811096         323.449018   
17   726954.255833  1.672975e+06        17      19.811096         343.260114   
18   726971.587812  1.672965e+06        18      19.811096         363.071210   
19   726988.919792  1.672956e+06        19      19.811096         382.882306   
20   727006.251771  1.672946e+06        20      19.811096         402.693401   
21   727023.583750  1.672937e+06        21      19.811096         422.504497   
22   727040.915729  1.672927e+06        22      19.811096         442.315593   
23   727058.247708  1.672917e+06        23      19.811096         462.126689   
24   727075.579688  1.672908e+06        24      19.811096         481.937785   
25   727092.911667  1.672898e+06        25      19.811096         501.748881   
26   727110.243646  1.672889e+06        26      19.811096         521.559976   
27   727127.575625  1.672879e+06        27      19.811096         541.371072   
28   727144.907604  1.672869e+06        28      19.811096         561.182168   
29   727162.239583  1.672860e+06        29      19.811096         580.993264   
..             ...           ...       ...            ...                ...   
126  728689.095000  1.671779e+06       126      19.480459        2495.395547   
127  728699.077143  1.671762e+06       127      19.480459        2514.876006   
128  728709.059286  1.671746e+06       128      19.480459        2534.356465   
129  728719.041429  1.671729e+06       129      19.480459        2553.836924   
130  728729.023571  1.671712e+06       130      19.480459        2573.317383   
131  728739.005714  1.671696e+06       131      19.480459        2592.797841   
132  728748.987857  1.671679e+06       132      19.480459        2612.278300   
133  728758.970000  1.671662e+06       133      19.638877        2631.917178   
134  728750.897727  1.671644e+06       134      19.638877        2651.556055   
135  728742.825455  1.671626e+06       135      19.638877        2671.194933   
136  728734.753182  1.671608e+06       136      19.638877        2690.833810   
137  728726.680909  1.671590e+06       137      19.638877        2710.472687   
138  728718.608636  1.671573e+06       138      19.638877        2730.111565   
139  728710.536364  1.671555e+06       139      19.638877        2749.750442   
140  728702.464091  1.671537e+06       140      19.638877        2769.389320   
141  728694.391818  1.671519e+06       141      19.638877        2789.028197   
142  728686.319545  1.671501e+06       142      19.638877        2808.667075   
143  728678.247273  1.671483e+06       143      19.638877        2828.305952   
144  728670

In [24]:
# add required field "z_dtm_align" to alignment_stationed_df 

# list of shapely geometry points                                                #ToDo JK: make df -> shp function           
alignment_stationed_geometry = ( 
    [sp.geometry.Point(row['x_align'], row['y_align']) for key, row in alignment_stationed_df.iterrows()] )
# create alignment_stationed_geometry_df
alignment_stationed_geometry_df = ( 
    gpd.GeoDataFrame(alignment_stationed_df, geometry=alignment_stationed_geometry, crs = crs) )
# write df to Alignment_stationed_shp (overwrite file)
alignment_stationed_geometry_df.to_file(Alignment_stationed_shp, driver='ESRI Shapefile') 

# get DTM values for alignment_points                                     #ToDo JK: make into what.points function
#   write to Alignment_dtm_csv
processing.runalg("grass7:r.what.points",DTM,Alignment_stationed_shp, "NA",",", 500,
                  True,False,False,False,False,grass_region,-1,0.0001,Alignment_dtm_csv)

# create alignment_dtm_df (dataframe) from Alignment_dtm_csv 
alignment_dtm_df = pd.read_csv(Alignment_dtm_csv)

# rename col=tmp... to "z_dtm_align"
alignment_dtm_df_col_tmp = [col for col in alignment_dtm_df.columns if 'tmp' in col]
if len(alignment_dtm_df_col_tmp) != 1:
    print "Extraction of DTM col=tmp did not work properly for alignment. Please check"
    exit()
alignment_dtm_df = alignment_dtm_df.rename(
    columns= {alignment_dtm_df_col_tmp[0]: "z_dtm_align"})

# write alignment_dtm_df["z_dtm_align"] to alignment_stationed_df["z_dtm_align"]
alignment_stationed_df["z_dtm_align"] = alignment_dtm_df["z_dtm_align"]

alignment_stationed_df = alignment_stationed_df.drop(columns = ["geometry"])

#alignment_stationed_df.head()
alignment_stationed_df

x_align       y_align  id_point  distance_stat  distance_stat_sum  \
0    726651.460000  1.673035e+06         0      18.328917          18.328917   
1    726669.697778  1.673037e+06         1      18.328917          36.657834   
2    726687.935556  1.673039e+06         2      18.328917          54.986750   
3    726706.173333  1.673041e+06         3      18.328917          73.315667   
4    726724.411111  1.673043e+06         4      18.328917          91.644584   
5    726742.648889  1.673044e+06         5      18.328917         109.973501   
6    726760.886667  1.673046e+06         6      18.328917         128.302418   
7    726779.124444  1.673048e+06         7      18.328917         146.631334   
8    726797.362222  1.673050e+06         8      18.328917         164.960251   
9    726815.600000  1.673052e+06         9      19.811096         184.771347   
10   726832.931979  1.673042e+06        10      19.811096         204.582443   
11   726850.263958  1.673032e+06        11      19.811096         224.393539   
12   726867.595938  1.673023e+06        12      19.811096         244.204635   
13   726884.927917  1.673013e+06        13      19.811096         264.015730   
14   726902.259896  1.673004e+06        14      19.811096         283.826826   
15   726919.591875  1.672994e+06        15      19.811096         303.637922   
16   726936.923854  1.672985e+06        16      19.811096         323.449018   
17   726954.255833  1.672975e+06        17      19.811096         343.260114   
18   726971.587812  1.672965e+06        18      19.811096         363.071210   
19   726988.919792  1.672956e+06        19      19.811096         382.882306   
20   727006.251771  1.672946e+06        20      19.811096         402.693401   
21   727023.583750  1.672937e+06        21      19.811096         422.504497   
22   727040.915729  1.672927e+06        22      19.811096         442.315593   
23   727058.247708  1.672917e+06        23      19.811096         462.126689   
24   727075.579688  1.672908e+06        24      19.811096         481.937785   
25   727092.911667  1.672898e+06        25      19.811096         501.748881   
26   727110.243646  1.672889e+06        26      19.811096         521.559976   
27   727127.575625  1.672879e+06        27      19.811096         541.371072   
28   727144.907604  1.672869e+06        28      19.811096         561.182168   
29   727162.239583  1.672860e+06        29      19.811096         580.993264   
..             ...           ...       ...            ...                ...   
126  728689.095000  1.671779e+06       126      19.480459        2495.395547   
127  728699.077143  1.671762e+06       127      19.480459        2514.876006   
128  728709.059286  1.671746e+06       128      19.480459        2534.356465   
129  728719.041429  1.671729e+06       129      19.480459        2553.836924   
130  728729.023571  1.671712e+06       130      19.480459        2573.317383   
131  728739.005714  1.671696e+06       131      19.480459        2592.797841   
132  728748.987857  1.671679e+06       132      19.480459        2612.278300   
133  728758.970000  1.671662e+06       133      19.638877        2631.917178   
134  728750.897727  1.671644e+06       134      19.638877        2651.556055   
135  728742.825455  1.671626e+06       135      19.638877        2671.194933   
136  728734.753182  1.671608e+06       136      19.638877        2690.833810   
137  728726.680909  1.671590e+06       137      19.638877        2710.472687   
138  728718.608636  1.671573e+06       138      19.638877        2730.111565   
139  728710.536364  1.671555e+06       139      19.638877        2749.750442   
140  728702.464091  1.671537e+06       140      19.638877        2769.389320   
141  728694.391818  1.671519e+06       141      19.638877        2789.028197   
142  728686.319545  1.671501e+06       142      19.638877        2808.667075   
143  728678.247273  1.671483e+06       143      19.638877        2828.305952   
144  728670

In [25]:
# Add require field "h" to alignment_stationed_df = overburden depth above station point 
alignment_stationed_df["h"] = alignment_stationed_df["z_dtm_align"] - alignment_stationed_df["z_align"] 

#alignment_stationed_df.head()
alignment_stationed_df

x_align       y_align  id_point  distance_stat  distance_stat_sum  \
0    726651.460000  1.673035e+06         0      18.328917          18.328917   
1    726669.697778  1.673037e+06         1      18.328917          36.657834   
2    726687.935556  1.673039e+06         2      18.328917          54.986750   
3    726706.173333  1.673041e+06         3      18.328917          73.315667   
4    726724.411111  1.673043e+06         4      18.328917          91.644584   
5    726742.648889  1.673044e+06         5      18.328917         109.973501   
6    726760.886667  1.673046e+06         6      18.328917         128.302418   
7    726779.124444  1.673048e+06         7      18.328917         146.631334   
8    726797.362222  1.673050e+06         8      18.328917         164.960251   
9    726815.600000  1.673052e+06         9      19.811096         184.771347   
10   726832.931979  1.673042e+06        10      19.811096         204.582443   
11   726850.263958  1.673032e+06        11      19.811096         224.393539   
12   726867.595938  1.673023e+06        12      19.811096         244.204635   
13   726884.927917  1.673013e+06        13      19.811096         264.015730   
14   726902.259896  1.673004e+06        14      19.811096         283.826826   
15   726919.591875  1.672994e+06        15      19.811096         303.637922   
16   726936.923854  1.672985e+06        16      19.811096         323.449018   
17   726954.255833  1.672975e+06        17      19.811096         343.260114   
18   726971.587812  1.672965e+06        18      19.811096         363.071210   
19   726988.919792  1.672956e+06        19      19.811096         382.882306   
20   727006.251771  1.672946e+06        20      19.811096         402.693401   
21   727023.583750  1.672937e+06        21      19.811096         422.504497   
22   727040.915729  1.672927e+06        22      19.811096         442.315593   
23   727058.247708  1.672917e+06        23      19.811096         462.126689   
24   727075.579688  1.672908e+06        24      19.811096         481.937785   
25   727092.911667  1.672898e+06        25      19.811096         501.748881   
26   727110.243646  1.672889e+06        26      19.811096         521.559976   
27   727127.575625  1.672879e+06        27      19.811096         541.371072   
28   727144.907604  1.672869e+06        28      19.811096         561.182168   
29   727162.239583  1.672860e+06        29      19.811096         580.993264   
..             ...           ...       ...            ...                ...   
126  728689.095000  1.671779e+06       126      19.480459        2495.395547   
127  728699.077143  1.671762e+06       127      19.480459        2514.876006   
128  728709.059286  1.671746e+06       128      19.480459        2534.356465   
129  728719.041429  1.671729e+06       129      19.480459        2553.836924   
130  728729.023571  1.671712e+06       130      19.480459        2573.317383   
131  728739.005714  1.671696e+06       131      19.480459        2592.797841   
132  728748.987857  1.671679e+06       132      19.480459        2612.278300   
133  728758.970000  1.671662e+06       133      19.638877        2631.917178   
134  728750.897727  1.671644e+06       134      19.638877        2651.556055   
135  728742.825455  1.671626e+06       135      19.638877        2671.194933   
136  728734.753182  1.671608e+06       136      19.638877        2690.833810   
137  728726.680909  1.671590e+06       137      19.638877        2710.472687   
138  728718.608636  1.671573e+06       138      19.638877        2730.111565   
139  728710.536364  1.671555e+06       139      19.638877        2749.750442   
140  728702.464091  1.671537e+06       140      19.638877        2769.389320   
141  728694.391818  1.671519e+06       141      19.638877        2789.028197   
142  728686.319545  1.671501e+06       142      19.638877        2808.667075   
143  728678.247273  1.671483e+06       143      19.638877        2828.305952   
144  728670

In [26]:
# define make_buffer to get buffer grid points at all station points along alignment
def make_buffer(point, overburden, c, res):
    h = overburden
    if h < 0.0:
        print "Overburden is negative. Please check"
        exit()
    intvls_r = max(int(h*c / res), 1)  #number of intervals along the buffer radius, close enough
    res_r = h*c / intvls_r  #effective resolution along the radius
    buffer = np.array(point)  #initialize buffer, first item is exactly at station point

    # calculate local coordinates for grid along a ring and add to point coor
    for i in range(intvls_r):
        r = c*h - i*res_r
        perim = 2 * r * pi 
        intvls_c = max(int(perim/res), 1)  #number of intervals along a ring, close enough
        item = np.array([0.0, 0.0])  #initialize       
        for j in range(intvls_c):
            item[0] = (sin((2*pi) / intvls_c *(j+1)) *r) + point[0]
            item[1] = (cos((2*pi) / intvls_c *(j+1)) *r) + point[1]
            buffer = np.vstack((buffer, item))  #vstack works with arrays of diff nr of items, append does not        

    return buffer

In [27]:
# create csv file with all buffer points     #ToDo KLK: make check plot of alignment, station points, buffer_all

# point = alignment_stationed_xy
# create alignment_stationed_xy from alignment_stationed_df with x,y of all station points
alignment_stationed_xy = alignment_stationed_df.as_matrix(columns=['x_align','y_align'])
# overburden = alignment_stationed_h
alignment_stationed_h = alignment_stationed_df.as_matrix(columns=['h'])

# initialize buffer_df, buffer_all_df, buffer_all
buffer_all = {}
buffer_df = pd.DataFrame(columns=["id_point", "x_align", "y_align", "z_align", "h" ,"x_buffer", "y_buffer"])
buffer_all_df = pd.DataFrame(columns=["id_point", "x_align", "y_align", "z_align", "h","x_buffer", "y_buffer"])

for n in range(0, len(alignment_stationed_df)): 
    buffer_point = make_buffer(point=alignment_stationed_xy[n], overburden=alignment_stationed_h[n], c=c, res=res)
    buffer_all[n] = buffer_df.copy(deep=False)  #copy of initialized buffer_df
    #print("n: ", n)
    #print(buffer_all)
    buffer_all[n]["id_point"] = [n] * len(buffer_point)  #list with len(buffer_point) number of n values) 
    buffer_all[n]["stat_sum"] = (  #make stat_sum correct for point n
        [alignment_stationed_df.iloc[n-1, alignment_stationed_df.columns.get_loc("distance_stat_sum")]] * len(buffer_point) ) 
    buffer_all[n]["dist_stat"] = (  #make stat_sum correct for point n
        [alignment_stationed_df.iloc[n-1, alignment_stationed_df.columns.get_loc("distance_stat")]] * len(buffer_point) ) 
    buffer_all[n]["x_align"] = ( 
        [alignment_stationed_df.iloc[n, alignment_stationed_df.columns.get_loc("x_align")]] * len(buffer_point) )      
    buffer_all[n]["y_align"] = ( 
        [alignment_stationed_df.iloc[n, alignment_stationed_df.columns.get_loc("y_align")]] * len(buffer_point) )      
    buffer_all[n]["z_align"] = ( 
        [alignment_stationed_df.iloc[n, alignment_stationed_df.columns.get_loc("z_align")]] * len(buffer_point) )           
    buffer_all[n]["h"] = ( 
        [alignment_stationed_df.iloc[n, alignment_stationed_df.columns.get_loc("h")]] * len(buffer_point) )           
    buffer_all[n]["x_buffer"] = buffer_point[0:,0]
    buffer_all[n]["y_buffer"] = buffer_point[0:,1]
    buffer_all_df = pd.concat([buffer_all_df, buffer_all[n]])
    #print(buffer_all_df)

# add variable "id_buffer_point" to buffer_all_df
buffer_all_df = buffer_all_df.reset_index(drop=True)
buffer_all_df["id_buffer_point"] = buffer_all_df.index    

# save buffer_all_df to csv
buffer_all_df.to_csv(Buffer_all_csv, sep=",", na_rep="NaN")

buffer_all_df.head()

dist_stat      h id_point  stat_sum    x_align       x_buffer     y_align  \
0        NaN  21.15        0       NaN  726651.46  726651.460000  1673035.25   
1        NaN  21.15        0       NaN  726651.46  726654.439322  1673035.25   
2        NaN  21.15        0       NaN  726651.46  726657.177277  1673035.25   
3        NaN  21.15        0       NaN  726651.46  726659.452052  1673035.25   
4        NaN  21.15        0       NaN  726651.46  726661.079358  1673035.25   

       y_buffer  z_align  id_buffer_point  
0  1.673035e+06   625.85                0  
1  1.673045e+06   625.85                1  
2  1.673044e+06   625.85                2  
3  1.673042e+06   625.85                3  
4  1.673040e+06   625.85                4

In [28]:
# add required field "z_dtm_buffer" and calcualted "dist" to buffer_all_df 
# add required field "slope" to buffer_all_df 

# buffer_all_df to Buffer_all_shp                                   #ToDo JK: reuse df -> shp function from above
# list of shapely geometry points
buffer_all_geometry = ( 
    [sp.geometry.Point(row['x_buffer'], row['y_buffer']) for key, row in buffer_all_df.iterrows()] )
# create buffer_all_geometry_df
buffer_all_geometry_df = gpd.GeoDataFrame(buffer_all_df, geometry=buffer_all_geometry, crs = crs)
# write df to Buffer_all_shp
buffer_all_geometry_df.to_file(Buffer_all_shp, driver='ESRI Shapefile') 
#print(buffer_all_geometry_df.head())

# get DTM values for Buffer_all_shp                             #ToDo JK: reuse what.points function from above
#   write to Buffer_dtm_csv
processing.runalg("grass7:r.what.points",DTM,Buffer_all_shp, "NA",",", 500,True,False,False,False,False,
                  grass_region,-1,0.0001,Buffer_dtm_csv)

# create buffer_dtm_df (dataframe) from Buffer_dtm_csv
buffer_dtm_df = pd.read_csv(Buffer_dtm_csv)

# rename col=tmp... to "z_dtm_buffer"
buffer_dtm_df_col_tmp = [col for col in buffer_dtm_df.columns if 'tmp' in col]
if len(buffer_dtm_df_col_tmp) != 1:
    print "Extraction of DTM col=tmp did not work properly for buffer. Please check"
    exit()
buffer_dtm_df = buffer_dtm_df.rename(
    columns= {buffer_dtm_df_col_tmp[0]: "z_dtm_buffer"})
#print(buffer_dtm_df.head())

# write buffer_dtm_df["z_dtm"] to buffer_all_df["z_dtm"]
buffer_all_df["z_dtm_buffer"] = buffer_dtm_df["z_dtm_buffer"]
#print(buffer_all_df.head())


# get slope values for Buffer_all_shp
#   write to Buffer_slope_csv
processing.runalg("grass7:r.what.points",DTM_slope,Buffer_all_shp, "NA",",", 500,True,False,False,False,False,
                  grass_region,-1,0.0001,Buffer_slope_csv)

# create buffer_dtm_df (dataframe) from Buffer_dtm_csv
buffer_slope_df = pd.read_csv(Buffer_slope_csv)

# rename col=tmp... to "z_dtm_buffer"
buffer_slope_df_col_tmp = [col for col in buffer_slope_df.columns if 'tmp' in col]
if len(buffer_slope_df_col_tmp) != 1:
    print "Extraction of Slope col=tmp did not work properly for buffer. Please check"
    exit()
buffer_slope_df = buffer_slope_df.rename(
    columns= {buffer_slope_df_col_tmp[0]: "slope"})
#print(buffer_slope_df.head())

# write buffer_dtm_df["z_dtm"] to buffer_all_df["z_dtm"]
buffer_all_df["slope"] = buffer_slope_df["slope"]


# calculate "dist" between each buffer point and associated alignment point 
buffer_all_df["dist"] = (((buffer_all_df["x_align"] - buffer_all_df["x_buffer"])**2 + 
                         (buffer_all_df["y_align"] - buffer_all_df["y_buffer"]) **2) +
                         (buffer_all_df["z_dtm_buffer"] - buffer_all_df["z_align"]) **2) **(0.5)

# clean up
buffer_all_df = buffer_all_df.drop(columns =["geometry"])
buffer_all_df.head()

dist_stat      h id_point  stat_sum    x_align       x_buffer     y_align  \
0        NaN  21.15        0       NaN  726651.46  726651.460000  1673035.25   
1        NaN  21.15        0       NaN  726651.46  726654.439322  1673035.25   
2        NaN  21.15        0       NaN  726651.46  726657.177277  1673035.25   
3        NaN  21.15        0       NaN  726651.46  726659.452052  1673035.25   
4        NaN  21.15        0       NaN  726651.46  726661.079358  1673035.25   

       y_buffer  z_align  id_buffer_point  z_dtm_buffer      slope       dist  
0  1.673035e+06   625.85                0         647.0  31.051294  21.150000  
1  1.673045e+06   625.85                1         644.0  36.716375  21.006026  
2  1.673044e+06   625.85                2         649.0  35.264390  25.450995  
3  1.673042e+06   625.85                3         649.0  35.264390  25.450995  
4  1.673040e+06   625.85                4         653.0  36.869898  29.136800

In [29]:
buffer_all_df

dist_stat          h id_point     stat_sum        x_align  \
0             NaN  21.150000        0          NaN  726651.460000   
1             NaN  21.150000        0          NaN  726651.460000   
2             NaN  21.150000        0          NaN  726651.460000   
3             NaN  21.150000        0          NaN  726651.460000   
4             NaN  21.150000        0          NaN  726651.460000   
5             NaN  21.150000        0          NaN  726651.460000   
6             NaN  21.150000        0          NaN  726651.460000   
7             NaN  21.150000        0          NaN  726651.460000   
8             NaN  21.150000        0          NaN  726651.460000   
9             NaN  21.150000        0          NaN  726651.460000   
10            NaN  21.150000        0          NaN  726651.460000   
11            NaN  21.150000        0          NaN  726651.460000   
12            NaN  21.150000        0          NaN  726651.460000   
13            NaN  21.150000        0          NaN  726651.460000   
14            NaN  21.150000        0          NaN  726651.460000   
15            NaN  21.150000        0          NaN  726651.460000   
16            NaN  21.150000        0          NaN  726651.460000   
17            NaN  21.150000        0          NaN  726651.460000   
18            NaN  21.150000        0          NaN  726651.460000   
19            NaN  21.150000        0          NaN  726651.460000   
20            NaN  21.150000        0          NaN  726651.460000   
21            NaN  21.150000        0          NaN  726651.460000   
22            NaN  21.150000        0          NaN  726651.460000   
23            NaN  21.150000        0          NaN  726651.460000   
24            NaN  21.150000        0          NaN  726651.460000   
25            NaN  21.150000        0          NaN  726651.460000   
26            NaN  21.150000        0          NaN  726651.460000   
27            NaN  21.150000        0          NaN  726651.460000   
28            NaN  21.150000        0          NaN  726651.460000   
29            NaN  21.150000        0          NaN  726651.460000   
...           ...        ...      ...          ...            ...   
476451  19.638877  34.528273      154  3024.694726  728589.452273   
476452  19.638877  34.528273      154  3024.694726  728589.452273   
476453  19.638877  34.528273      154  3024.694726  728589.452273   
476454  19.638877  34.528273      154  3024.694726  728589.452273   
476455  19.638877  34.528273      154  3024.694726  728589.452273   
476456  19.638877  34.528273      154  3024.694726  728589.452273   
476457  19.638877  34.528273      154  3024.694726  728589.452273   
476458  19.638877  34.528273      154  3024.694726  728589.452273   
476459  19.638877  34.528273      154  3024.694726  728589.452273   
476460  19.638877  34.528273      154  3024.694726  728589.452273   
476461  19.638877  34.528273      154  3024.694726  728589.452273   
476462  19.638877  34.528273      154  3024.694726  728589.452273   
476463  19.638877  34.528273      154  3024.694726  728589.452273   
476464  19.638877  34.528273      154  3024.694726  728589.452273   
476465  19.638877  34.528273      154  3024.694726  728589.452273   
476466  19.638877  34.528273      154  3024.694726  728589.452273   
476467  19.638877  34.528273      154  3024.694726  728589.452273   
476468  19.638877  34.528273      154  3024.694726  728589.452273   
476469  19.638877  34.528273      154  3024.694726  728589.452273   
476470  19.638877  34.528273      154  3024.694726  728589.452273   
476471  19.638877   8.780000      155  3044.333604  728581.380000   
476472  19.638877   8.780000      155  3044.333604  728581.380000   
476473  19.638877   8.780000      155  3044.333604  728581.380000   
476474  19.638877   8.780000      155  3044.333604  728581.380000   
476475  19.638877   8.780000      155  3044.333604  728581.380000   
476476  19.638877   8.780000      155  3044.333604  728581.380000   
476477

In [30]:
# calculate minimum distance to terrain in each buffer ring
                                                #ToDo KLK: make check plot of id_buffer_points with "min_dist"
                                                #ToDo JK: add station field
buffer_all_df["min_dist"] = np.nan

for n in range(0, len(alignment_stationed_df)):
    buffer_all_df_sel = buffer_all_df.loc[(buffer_all_df["id_point"] == n),]
    dist_idxmin=buffer_all_df_sel['dist'].idxmin()
    buffer_all_df.loc[(buffer_all_df["id_buffer_point"] == dist_idxmin), "min_dist"] = "MIN"

buffer_all_df.to_csv(Buffer_all_csv, header=True, index=False)  #no header
#buffer_all_df

buffer_all_df.loc[(buffer_all_df["min_dist"] == "MIN"),]

dist_stat           h id_point     stat_sum        x_align  \
32            NaN   21.150000        0          NaN  726651.460000   
69      18.328917   31.184111        1    18.328917  726669.697778   
215     18.328917   43.218222        2    36.657834  726687.935556   
413     18.328917   57.252333        3    54.986750  726706.173333   
851     18.328917   70.286444        4    73.315667  726724.411111   
1117    18.328917   86.320556        5    91.644584  726742.648889   
2142    18.328917   96.354667        6   109.973501  726760.886667   
2655    18.328917  100.388778        7   128.302418  726779.124444   
3622    18.328917  106.422889        8   146.631334  726797.362222   
5207    18.328917  108.457000        9   164.960251  726815.600000   
6369    19.811096  102.720719       10   184.771347  726832.931979   
7223    19.811096   96.984437       11   204.582443  726850.263958   
7852    19.811096   93.248156       12   224.393539  726867.595938   
8681    19.811096   92.511875       13   244.204635  726884.927917   
9270    19.811096   92.775594       14   264.015730  726902.259896   
9882    19.811096   91.039312       15   283.826826  726919.591875   
10559   19.811096   92.303031       16   303.637922  726936.923854   
11320   19.811096   90.566750       17   323.449018  726954.255833   
12215   19.811096   84.830469       18   343.260114  726971.587812   
12964   19.811096   78.094188       19   363.071210  726988.919792   
13712   19.811096   84.357906       20   382.882306  727006.251771   
14220   19.811096   93.621625       21   402.693401  727023.583750   
14876   19.811096  104.885344       22   422.504497  727040.915729   
16074   19.811096  113.149062       23   442.315593  727058.247708   
16749   19.811096  117.412781       24   462.126689  727075.579688   
17976   19.811096  118.676500       25   481.937785  727092.911667   
19958   19.811096  114.940219       26   501.748881  727110.243646   
21011   19.811096  106.203937       27   521.559976  727127.575625   
22013   19.811096   99.467656       28   541.371072  727144.907604   
23043   19.811096   98.731375       29   561.182168  727162.239583   
...           ...         ...      ...          ...            ...   
374928  19.480459  230.625000      126  2475.915088  728689.095000   
379986  19.480459  235.856000      127  2495.395547  728699.077143   
385298  19.480459  248.087000      128  2514.876006  728709.059286   
391250  19.480459  264.318000      129  2534.356465  728719.041429   
397511  19.480459  274.549000      130  2553.836924  728729.023571   
403854  19.480459  277.780000      131  2573.317383  728739.005714   
409951  19.480459  280.011000      132  2592.797841  728748.987857   
413556  19.480459  277.242000      133  2612.278300  728758.970000   
420341  19.638877  268.493727      134  2631.917178  728750.897727   
426894  19.638877  259.745455      135  2651.556055  728742.825455   
432319  19.638877  249.997182      136  2671.194933  728734.753182   
438042  19.638877  243.248909      137  2690.833810  728726.680909   
444141  19.638877  235.500636      138  2710.472687  728718.608636   
448134  19.638877  224.752364      139  2730.111565  728710.536364   
452808  19.638877  216.004091      140  2749.750442  728702.464091   
456934  19.638877  201.255818      141  2769.389320  728694.391818   
460660  19.638877  190.507545      142  2789.028197  728686.319545   
463937  19.638877  174.759273      143  2808.667075  728678.247273   
466883  19.638877  162.011000      144  2828.305952  728670.175000   
468768  19.638877  147.262727      145  2847.944829  728662.102727   
470630  19.638877  132.514455      146  2867.583707  728654.030455   
472062  19.638877  120.766182      147  2887.222584  728645.958182   
473521  19.638877  105.017909      148  2906.861462  728637.885909   
474111  19.638877   93.269636      149  2926.500339  728629.813636   
474880  19.638877   82.521364      150  2946.139216  728621.741364   
475614  19.638877 

In [31]:
# calculate hydraulic confinement safety factor at each station point
#   required input data: reference maximum water pressure elevation (static or dynamic ??) 


buffer_all_df_sel = buffer_all_df.loc[(buffer_all_df["min_dist"] == "MIN"),]


dist = array(buffer_all_df_sel['dist'])
slope = array(buffer_all_df_sel['slope'])
z_align = array(buffer_all_df_sel['z_align'])
stat_sum = array(buffer_all_df_sel['stat_sum'])
stat_sum[0] = 0  #correction for station_stat_sum being for n-1 above (to be fixed above)

def hydr_conf_sf(density_rock, max_static_level, z_align, min_dist, slope):
  density_water = 9.805
  static_head = max_static_level - z_align 
  sf = (min_dist * density_rock *  cos(slope*pi/180.)) / (static_head * density_water)
  return sf

FS = hydr_conf_sf(28.0, max_static_water_level, z_align, dist, slope)
print FS

buffer_all_df_sel["FS"] = np.nan
for n in range(0, len(buffer_all_df_sel)):
    buffer_all_df_sel.iloc[n, buffer_all_df_sel.columns.get_loc("FS")] = FS[n]

buffer_all_df_sel = buffer_all_df_sel.drop(columns =["x_align", "y_align", "min_dist", "id_buffer_point"])
    

[ 3.11088222  5.29042471  7.50172766  9.67727522 11.6416769  13.76489876
 15.03055554 16.74081695 21.42140469 23.58236305 20.20828479 18.46076209
 16.23210634 13.69084068 12.16554208 10.55299186  9.46552544  8.33117218
  8.24937018  8.41074325  8.45022012  8.61670351  8.3625347   8.87535184
  8.03841766  7.81578443  7.56676599  7.60806186  7.40615666  7.27413564
  7.22929499  7.1942914   7.23006151  7.404456    7.39353482  7.63699304
  8.29513801  8.41280566  8.00023566  7.20019237  6.73608737  6.72624394
  6.52704526  6.49494545  6.27471531  6.29048562  6.2182622   6.26438263
  6.79268997  6.45856768  6.65340533  7.28823068  7.13925445  6.28827902
  6.07298353  6.09747617  5.89631218  5.7869817   5.67232709  5.52972925
  5.52745801  5.57980169  5.82076771  5.63809164  5.87495138  5.40207576
  5.28191764  5.26359993  5.18037013  5.23412433  5.25021429  5.26004445
  5.32199871  5.43508327  5.32166184  5.45940932  5.48177769  6.05344018
  6.51659167  6.48487137  6.42347874  6.36335237  6

In [32]:
buffer_all_df_sel

dist_stat           h id_point     stat_sum       x_buffer  \
32            NaN   21.150000        0          NaN  726644.586758   
69      18.328917   31.184111        1    18.328917  726654.405319   
215     18.328917   43.218222        2    36.657834  726669.476723   
413     18.328917   57.252333        3    54.986750  726684.656736   
851     18.328917   70.286444        4    73.315667  726699.400472   
1117    18.328917   86.320556        5    91.644584  726699.725048   
2142    18.328917   96.354667        6   109.973501  726724.761591   
2655    18.328917  100.388778        7   128.302418  726729.876301   
3622    18.328917  106.422889        8   146.631334  726797.362222   
5207    18.328917  108.457000        9   164.960251  726836.091526   
6369    19.811096  102.720719       10   184.771347  726845.474884   
7223    19.811096   96.984437       11   204.582443  726865.487481   
7852    19.811096   93.248156       12   224.393539  726882.018271   
8681    19.811096   92.511875       13   244.204635  726892.406643   
9270    19.811096   92.775594       14   264.015730  726903.764613   
9882    19.811096   91.039312       15   283.826826  726916.571979   
10559   19.811096   92.303031       16   303.637922  726922.191423   
11320   19.811096   90.566750       17   323.449018  726945.271871   
12215   19.811096   84.830469       18   343.260114  726958.310596   
12964   19.811096   78.094188       19   363.071210  726974.795911   
13712   19.811096   84.357906       20   382.882306  726988.321490   
14220   19.811096   93.621625       21   402.693401  726989.331404   
14876   19.811096  104.885344       22   422.504497  726994.741857   
16074   19.811096  113.149062       23   442.315593  727014.245295   
16749   19.811096  117.412781       24   462.126689  727081.615937   
17976   19.811096  118.676500       25   481.937785  727083.926207   
19958   19.811096  114.940219       26   501.748881  727126.219508   
21011   19.811096  106.203937       27   521.559976  727137.943249   
22013   19.811096   99.467656       28   541.371072  727141.910229   
23043   19.811096   98.731375       29   561.182168  727154.985478   
...           ...         ...      ...          ...            ...   
374928  19.480459  230.625000      126  2475.915088  728648.777264   
379986  19.480459  235.856000      127  2495.395547  728648.275821   
385298  19.480459  248.087000      128  2514.876006  728647.854507   
391250  19.480459  264.318000      129  2534.356465  728649.958315   
397511  19.480459  274.549000      130  2553.836924  728653.388051   
403854  19.480459  277.780000      131  2573.317383  728654.733494   
409951  19.480459  280.011000      132  2592.797841  728654.381382   
413556  19.480459  277.242000      133  2612.278300  728805.976430   
420341  19.638877  268.493727      134  2631.917178  728803.525967   
426894  19.638877  259.745455      135  2651.556055  728796.554204   
432319  19.638877  249.997182      136  2671.194933  728796.599848   
438042  19.638877  243.248909      137  2690.833810  728796.867964   
444141  19.638877  235.500636      138  2710.472687  728785.105106   
448134  19.638877  224.752364      139  2730.111565  728765.205857   
452808  19.638877  216.004091      140  2749.750442  728706.975978   
456934  19.638877  201.255818      141  2769.389320  728704.922234   
460660  19.638877  190.507545      142  2789.028197  728686.319545   
463937  19.638877  174.759273      143  2808.667075  728691.739848   
466883  19.638877  162.011000      144  2828.305952  728686.603296   
468768  19.638877  147.262727      145  2847.944829  728677.069567   
470630  19.638877  132.514455      146  2867.583707  728670.415150   
472062  19.638877  120.766182      147  2887.222584  728662.319985   
473521  19.638877  105.017909      148  2906.861462  728646.820994   
474111  19.638877   93.269636      149  2926.500339  728637.332611   
474880  19.638877   82.521364      150  2946.139216  728636.480954   
475614  19.638877 

In [33]:
# calculate hydraulic confinement safety factor at each station point
#   required input data: reference maximum water pressure elevation (static or dynamic ??) 


buffer_all_df_sel = buffer_all_df.loc[(buffer_all_df["min_dist"] == "MIN"),]


dist = array(buffer_all_df_sel['dist'])
slope = array(buffer_all_df_sel['slope'])
z_align = array(buffer_all_df_sel['z_align'])
stat_sum = array(buffer_all_df_sel['stat_sum'])
stat_sum[0] = 0  #correction for station_stat_sum being for n-1 above (to be fixed above)

FS = (dist * 28.0 * cos(slope*pi/180.)) / ((637.20 - z_align) * 10)
print(FS)
print(stat_sum)

buffer_all_df_sel["FS"] = np.nan
for n in range(0, len(buffer_all_df_sel)):
    buffer_all_df_sel.iloc[n, buffer_all_df_sel.columns.get_loc("FS")] = FS[n]

buffer_all_df_sel = buffer_all_df_sel.drop(columns =["x_align", "y_align", "min_dist", "id_buffer_point"])
    

[ 3.05022002  5.18726143  7.35544397  9.48856836 11.4146642  13.49648324
 14.73745971 16.41437102 21.0036873  23.12250697 19.81422324 18.10077723
 15.91558027 13.42386929 11.92831401 10.34720852  9.28094769  8.16871432
  8.08850746  8.24673375  8.28544083  8.44867779  8.19946527  8.70228248
  7.88166852  7.66337664  7.41921406  7.45970466  7.2617366   7.13229
  7.08832374  7.05400272  7.08907531  7.26006911  7.24936089  7.48807168
  8.13338282  8.24875595  7.84423106  7.05978862  6.60473366  6.59508218
  6.39976788  6.36829401  6.15235837  6.16782115  6.09700608  6.14222717
  6.66023251  6.33262561  6.52366393  7.14611018  7.00003899  6.16565758
  5.95456035  5.97857538  5.7813341   5.67413556  5.56171671  5.42189953
  5.41967258  5.47099556  5.70726274  5.52814885  5.76038983  5.29673528
  5.17892025  5.16095973  5.07935291  5.1320589   5.14783511  5.15747358
  5.21821974  5.32909915  5.21788944  5.35295083  5.37488303  5.93539809
  6.38951813  6.35841637  6.2982209   6.239267    6.19

In [34]:
#buffer_all_df_sel["stat_sum"]
#buffer_all_df_sel.iloc[1, buffer_all_df_sel.columns.get_loc("stat_sum")]
buffer_all_df_sel.iloc[1, buffer_all_df_sel.columns.get_loc("FS")]


5.187261428261646

In [35]:
buffer_all_df_sel

dist_stat           h id_point     stat_sum       x_buffer  \
32            NaN   21.150000        0          NaN  726644.586758   
69      18.328917   31.184111        1    18.328917  726654.405319   
215     18.328917   43.218222        2    36.657834  726669.476723   
413     18.328917   57.252333        3    54.986750  726684.656736   
851     18.328917   70.286444        4    73.315667  726699.400472   
1117    18.328917   86.320556        5    91.644584  726699.725048   
2142    18.328917   96.354667        6   109.973501  726724.761591   
2655    18.328917  100.388778        7   128.302418  726729.876301   
3622    18.328917  106.422889        8   146.631334  726797.362222   
5207    18.328917  108.457000        9   164.960251  726836.091526   
6369    19.811096  102.720719       10   184.771347  726845.474884   
7223    19.811096   96.984437       11   204.582443  726865.487481   
7852    19.811096   93.248156       12   224.393539  726882.018271   
8681    19.811096   92.511875       13   244.204635  726892.406643   
9270    19.811096   92.775594       14   264.015730  726903.764613   
9882    19.811096   91.039312       15   283.826826  726916.571979   
10559   19.811096   92.303031       16   303.637922  726922.191423   
11320   19.811096   90.566750       17   323.449018  726945.271871   
12215   19.811096   84.830469       18   343.260114  726958.310596   
12964   19.811096   78.094188       19   363.071210  726974.795911   
13712   19.811096   84.357906       20   382.882306  726988.321490   
14220   19.811096   93.621625       21   402.693401  726989.331404   
14876   19.811096  104.885344       22   422.504497  726994.741857   
16074   19.811096  113.149062       23   442.315593  727014.245295   
16749   19.811096  117.412781       24   462.126689  727081.615937   
17976   19.811096  118.676500       25   481.937785  727083.926207   
19958   19.811096  114.940219       26   501.748881  727126.219508   
21011   19.811096  106.203937       27   521.559976  727137.943249   
22013   19.811096   99.467656       28   541.371072  727141.910229   
23043   19.811096   98.731375       29   561.182168  727154.985478   
...           ...         ...      ...          ...            ...   
374928  19.480459  230.625000      126  2475.915088  728648.777264   
379986  19.480459  235.856000      127  2495.395547  728648.275821   
385298  19.480459  248.087000      128  2514.876006  728647.854507   
391250  19.480459  264.318000      129  2534.356465  728649.958315   
397511  19.480459  274.549000      130  2553.836924  728653.388051   
403854  19.480459  277.780000      131  2573.317383  728654.733494   
409951  19.480459  280.011000      132  2592.797841  728654.381382   
413556  19.480459  277.242000      133  2612.278300  728805.976430   
420341  19.638877  268.493727      134  2631.917178  728803.525967   
426894  19.638877  259.745455      135  2651.556055  728796.554204   
432319  19.638877  249.997182      136  2671.194933  728796.599848   
438042  19.638877  243.248909      137  2690.833810  728796.867964   
444141  19.638877  235.500636      138  2710.472687  728785.105106   
448134  19.638877  224.752364      139  2730.111565  728765.205857   
452808  19.638877  216.004091      140  2749.750442  728706.975978   
456934  19.638877  201.255818      141  2769.389320  728704.922234   
460660  19.638877  190.507545      142  2789.028197  728686.319545   
463937  19.638877  174.759273      143  2808.667075  728691.739848   
466883  19.638877  162.011000      144  2828.305952  728686.603296   
468768  19.638877  147.262727      145  2847.944829  728677.069567   
470630  19.638877  132.514455      146  2867.583707  728670.415150   
472062  19.638877  120.766182      147  2887.222584  728662.319985   
473521  19.638877  105.017909      148  2906.861462  728646.820994   
474111  19.638877   93.269636      149  2926.500339  728637.332611   
474880  19.638877   82.521364      150  2946.139216  728636.480954   
475614  19.638877 

In [182]:
z_dtm_align = go.Scatter(
    x=alignment_stationed_df['distance_stat_sum'].tolist(),
    y=alignment_stationed_df['z_dtm_align'].tolist(),
    name = "Terrain",
    mode='lines',
    line=dict(width=0.5,
              color='rgb(196, 97, 26)'),
    fill='tonexty'
)

z_align_line = go.Scatter(
    x=alignment_stationed_df['distance_stat_sum'].tolist(),
    y=alignment_stationed_df['z_align'].tolist(),
    line=dict(width=3,
              color='rgb(0, 0, 0)'),
    name ="Tunnel"
)

data = [z_dtm_align, z_align_line]

layout = go.Layout(
    title='Longitudinal Section',
    showlegend=False,
    xaxis=dict(
        title= "Stationing",
    ),
    yaxis=dict(
        title = "m a.s.l.",
        range = [300,800]  #nees a procedure to automate this
    ),
    margin=dict(
        #l=50,
        b=40
    ),
    height=400,
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, show_link=False, filename='stacked-area-plot')



# initialize variables 
x_data = []
y_data = []
traces = []
annotations = []

fs = buffer_all_df_sel['FS'].tolist(),
c_fs = []
for i in range(len(buffer_all_df_sel.index)):
    if fs[0][i] <= 1.4:
        c_fs.append('red')
    elif 1.4 < fs[0][i] < 1.8:
        c_fs.append('yellow')
    else:
        c_fs.append('green')
slope = buffer_all_df_sel['slope'].tolist(),
c_slope = []
for i in range(len(buffer_all_df_sel.index)):
    if slope[0][i] > 30:
        c_slope.append('red')
    else:
        c_slope.append('green')        

x_data = [buffer_all_df_sel['dist_stat'].tolist()]
# x_data = [buffer_all_df_sel['dist_stat'].tolist(),
#           buffer_all_df_sel['dist_stat'].tolist()]

y_data = ['hydraulic confinement']
# y_data = ['slope',
#           'hydraulic confinement']

for i in range(0, len(x_data[0])):
    for xd, yd in zip(x_data, y_data):
        traces.append(go.Bar(
            x=[xd[i]],
            y=[yd],
            width = 0.3,
            orientation='h',
            marker=dict(
                color = c_fs[i],
            ),
            hoverinfo = 'none',
        ))
        
layout = go.Layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.08, 1]  #horizontal extent of bar
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    #paper_bgcolor='rgb(248, 248, 255)',
    #plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(
        l=0,
        r=0,
        t=0,
        b=0
    ),
    showlegend=False,
    width=945,
    height=70,
)

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.09, y=yd, #x is position of text
                            xanchor='left',
                            text=str(yd),
                            font=dict(family='Arial', size=14,
                                      #color='rgb(67, 67, 67)'),
                                      color='white'),
                            showarrow=False, align='left'))
layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
plotly.offline.iplot(fig, show_link=False, config=dict(displayModeBar=False), filename='bar-colorscale')

In [200]:
z_dtm_align = go.Scatter(
    x=alignment_stationed_df['distance_stat_sum'].tolist(),
    y=alignment_stationed_df['z_dtm_align'].tolist(),
    name = "Terrain",
    mode='lines',
    line=dict(width=0.5,
              color='rgb(196, 97, 26)'),
    fill='tonexty'
)

z_align_line = go.Scatter(
    x=alignment_stationed_df['distance_stat_sum'].tolist(),
    y=alignment_stationed_df['z_align'].tolist(),
    line=dict(width=3,
              color='rgb(0, 0, 0)'),
    name ="Tunnel"
)

data = [z_dtm_align, z_align_line]

layout = go.Layout(
    title='Longitudinal Section',
    showlegend=False,
    xaxis=dict(
        title= "Stationing",
    ),
    yaxis=dict(
        title = "m a.s.l.",
        range = [300,800]  #nees a procedure to automate this
    ),
    margin=dict(
        #l=50,
        b=40
    ),
    height=400,
)

fig = go.Figure(data=data, layout=layout)
plotly.offline.iplot(fig, show_link=False, filename='stacked-area-plot')



# initialize variables 
x_data = []
y_data = []
traces = []
annotations = []

fs = buffer_all_df_sel['FS'].tolist(),
c_fs = []
for i in range(len(buffer_all_df_sel.index)):
    if fs[0][i] <= 1.4:
        c_fs.append('red')
    elif 1.4 < fs[0][i] < 1.8:
        c_fs.append('yellow')
    else:
        c_fs.append('green')
slope = buffer_all_df_sel['slope'].tolist(),
c_slope = []
for i in range(len(buffer_all_df_sel.index)):
    if slope[0][i] > 30:
        c_slope.append('red')
    else:
        c_slope.append('green')        
h = buffer_all_df_sel['h'].tolist(),
c_h = []
for i in range(len(buffer_all_df_sel.index)):
    if h[0][i] > 200:
        c_h.append('orange')
    elif h[0][i] < 15:
        c_h.append('rgb(191, 34, 186)')
    else:
        c_h.append('green')        

x_data = [buffer_all_df_sel['dist_stat'].tolist(),
          buffer_all_df_sel['dist_stat'].tolist(),
          buffer_all_df_sel['dist_stat'].tolist()]

y_data = ['overburden',
          'hillslope',
          'hydraulic confinement']

for i in range(0, len(x_data[0])):
    n = 0
    for xd, yd in zip(x_data, y_data):
        n = n +1
        if n == 1:
            traces.append(go.Bar(
                x=[xd[i]],
                y=[yd],
                width = 0.6,
                orientation='h',
                marker=dict(
                    color = c_h[i],
                ),
                hoverinfo = 'none',
            ))
        elif n == 2:
            traces.append(go.Bar(
                x=[xd[i]],
                y=[yd],
                width = 0.6,
                orientation='h',
                marker=dict(
                    color = c_slope[i],
                ),
                hoverinfo = 'none',
            ))            
        elif n == 3:
            traces.append(go.Bar(
                x=[xd[i]],
                y=[yd],
                width = 0.6,
                orientation='h',
                marker=dict(
                    color = c_fs[i],
                ),
                hoverinfo = 'none',
            ))            
        
layout = go.Layout(
    xaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
        domain=[0.08, 1]  #horizontal extent of bar
    ),
    yaxis=dict(
        showgrid=False,
        showline=False,
        showticklabels=False,
        zeroline=False,
    ),
    barmode='stack',
    #paper_bgcolor='rgb(248, 248, 255)',
    #plot_bgcolor='rgb(248, 248, 255)',
    margin=dict(
        l=0,
        r=0,
        t=0,
        b=0
    ),
    showlegend=False,
    width=945,
    height=105,
)

for yd, xd in zip(y_data, x_data):
    # labeling the y-axis
    annotations.append(dict(xref='paper', yref='y',
                            x=0.09, y=yd, #x is position of text
                            xanchor='left',
                            text=str(yd),
                            font=dict(family='Arial', size=16,
                                      #color='rgb(67, 67, 67)'),
                                      color='white'),
                            showarrow=False, align='left'))
layout['annotations'] = annotations

fig = go.Figure(data=traces, layout=layout)
plotly.offline.iplot(fig, show_link=False, config=dict(displayModeBar=False), filename='bar-colorscale')

In [148]:
alignment_stationed_line_plot = go.Scatter(
    x=alignment_stationed_df['x_align'].tolist(),
    y=alignment_stationed_df['y_align'].tolist(),
    line=dict(width=3,
              color='rgb(0, 0, 0)'),
    name ="alignment_stationed_line"
)

alignment_point_plot = go.Scatter(
    x=alignment_df['Easting'].tolist(),
    y=alignment_df['Northing'].tolist(),
    name ="alignment_point",
    mode = "markers",
    marker = dict(
        size = 20,
        color = 'rgb(255, 153, 153)'
    )
)

alignment_stationed_point_plot = go.Scatter(
    x=alignment_stationed_df['x_align'].tolist(),
    y=alignment_stationed_df['y_align'].tolist(),
    mode = "markers",
    name ="alignment_stationed_point",
    marker = dict(
        size = 5, #15,
        color = 'rgb(255, 0, 0)'
    )
)

# buffer_all_df_point_plot = go.Scatter(
#     x=buffer_all_df['x_buffer'].tolist(),
#     y=buffer_all_df['y_buffer'].tolist(),
#     mode = "markers",
#     name ="buffer_all_point",
#      marker = dict(
#         #size = 10,
#         color = 'rgb(0, 0, 153)'
#     )
# )
buffer_all_df_point_plot = go.Scatter(
    x=buffer_all_df_sel['x_buffer'].tolist(),
    y=buffer_all_df_sel['y_buffer'].tolist(),
    mode = "markers",
    name ="buffer_min",
     marker = dict(
        #size = 10,
        color = 'rgb(0, 0, 153)'
    )
)

data = [alignment_stationed_line_plot, alignment_point_plot, alignment_stationed_point_plot, buffer_all_df_point_plot]



fig = go.Figure(data=data)
plotly.offline.iplot(fig, show_link=False, filename='stacked-area-plot')

In [ ]:
alignment_df.head()

In [ ]:
# plot results for hydraulic confinement safety factor as horizontal bar beneath longitudinal profile 



